## Using wrappers for Scikit learn API

This tutorial is about using gensim models as a part of your scikit learn workflow with the help of wrappers found at ```gensim.sklearn_integration.sklearn_wrapper_gensim_ldaModel```

The wrapper available (as of now) are :
* LdaModel (```gensim.sklearn_integration.sklearn_wrapper_gensim_ldaModel.SklearnWrapperLdaModel```),which implements gensim's ```LdaModel``` in a scikit-learn interface

### LdaModel

To use LdaModel begin with importing LdaModel wrapper

In [3]:
import sys
sys.path.insert(0, '/home/kris/Desktop/GsoC2K17/gensim/gensim/')

In [4]:
import gensim
print gensim.__version__

1.0.1


In [5]:
from gensim.sklearn_integration.sklearn_wrapper_gensim_ldamodel import SklearnWrapperLdaModel

Next we will create a dummy set of texts and convert it into a corpus

In [6]:
from gensim.corpora import Dictionary
texts = [['complier', 'system', 'computer'],
 ['eulerian', 'node', 'cycle', 'graph', 'tree', 'path'],
 ['graph', 'flow', 'network', 'graph'],
 ['loading', 'computer', 'system'],
 ['user', 'server', 'system'],
 ['tree','hamiltonian'],
 ['graph', 'trees'],
 ['computer', 'kernel', 'malfunction','computer'],
 ['server','system','computer']]
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

Then to run the LdaModel on it

In [7]:
model=SklearnWrapperLdaModel(num_topics=2,id2word=dictionary,iterations=20, random_state=1)
model.fit(corpus)
model.print_topics(2)

[(0,
  u'0.164*"computer" + 0.117*"system" + 0.105*"graph" + 0.061*"server" + 0.057*"tree" + 0.046*"malfunction" + 0.045*"kernel" + 0.045*"complier" + 0.043*"loading" + 0.039*"hamiltonian"'),
 (1,
  u'0.102*"graph" + 0.083*"system" + 0.072*"tree" + 0.064*"server" + 0.059*"user" + 0.059*"computer" + 0.057*"trees" + 0.056*"eulerian" + 0.055*"node" + 0.052*"flow"')]

### Integration with Sklearn

To provide a better example of how it can be used with Sklearn, Let's use CountVectorizer method of sklearn. For this example we will use [20 Newsgroups data set](http://qwone.com/~jason/20Newsgroups/). We will only use the categories rec.sport.baseball and sci.crypt and use it to generate topics.

In [8]:
import numpy as np
from gensim import matutils
from gensim.models.ldamodel import LdaModel
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from gensim.sklearn_integration.sklearn_wrapper_gensim_ldamodel import SklearnWrapperLdaModel

In [9]:
rand = np.random.mtrand.RandomState(1) # set seed for getting same result
cats = ['rec.sport.baseball', 'sci.crypt']
data = fetch_20newsgroups(subset='train',
                        categories=cats,
                        shuffle=True)

Next, we use countvectorizer to convert the collection of text documents to a matrix of token counts.

In [38]:
vec = CountVectorizer(min_df=10, stop_words='english')

X = vec.fit_transform(data.data)
vocab = vec.get_feature_names() #vocab to be converted to id2word 

id2word=dict([(i, s) for i, s in enumerate(vocab)])

Next, we just need to fit X and id2word to our Lda wrapper.

In [61]:
obj=SklearnWrapperLdaModel(id2word=id2word,num_topics=5,passes=20)
lda=obj.fit(X)
lda.print_topics()

here


[(0,
  u'0.061*"ciphertext" + 0.028*"abroad" + 0.027*"amolitor" + 0.022*"brian" + 0.020*"agree" + 0.016*"argue" + 0.013*"facts" + 0.010*"crypto" + 0.010*"asking" + 0.009*"capabilities"'),
 (1,
  u'0.031*"accurate" + 0.022*"corporate" + 0.019*"clark" + 0.017*"decipher" + 0.017*"example" + 0.015*"cases" + 0.015*"follow" + 0.015*"basically" + 0.015*"consideration" + 0.013*"authentication"'),
 (2,
  u'0.014*"face" + 0.013*"bitnet" + 0.011*"constitutional" + 0.011*"false" + 0.010*"digex" + 0.008*"abuse" + 0.008*"effort" + 0.008*"costs" + 0.008*"breaking" + 0.007*"cover"'),
 (3,
  u'0.091*"abroad" + 0.079*"asking" + 0.059*"cryptography" + 0.032*"certain" + 0.030*"arithmetic" + 0.027*"69" + 0.026*"book" + 0.025*"87" + 0.024*"cracking" + 0.024*"demand"'),
 (4,
  u'0.014*"cryptography" + 0.013*"corporate" + 0.013*"chance" + 0.010*"accurate" + 0.008*"dawson" + 0.006*"afford" + 0.005*"broad" + 0.005*"clark" + 0.005*"brett" + 0.005*"entirely"')]

#### Using together with Scikit learn's Logistic Regression

Now lets try Sklearn's logistic classifier to classify the given categories into two types.Ideally we should get postive weights when cryptography is talked about and negative when baseball is talked about.

In [21]:
from sklearn import linear_model

In [22]:
def print_features(clf, vocab, n=10):
    ''' Better printing for sorted list '''
    coef = clf.coef_[0]
    print 'Positive features: %s' % (' '.join(['%s:%.2f' % (vocab[j], coef[j]) for j in np.argsort(coef)[::-1][:n] if coef[j] > 0]))
    print 'Negative features: %s' % (' '.join(['%s:%.2f' % (vocab[j], coef[j]) for j in np.argsort(coef)[:n] if coef[j] < 0]))

In [10]:
clf=linear_model.LogisticRegression(penalty='l1', C=0.1) #l1 penalty used
clf.fit(X,data.target)
print_features(clf,vocab)

Positive features: clipper:1.50 code:1.24 key:1.04 encryption:0.95 chip:0.37 nsa:0.37 government:0.36 uk:0.36 org:0.23 cryptography:0.23
Negative features: baseball:-1.32 game:-0.71 year:-0.61 team:-0.38 edu:-0.27 games:-0.26 players:-0.23 ball:-0.17 season:-0.14 phillies:-0.11


### Example for Using Grid Search

In [8]:
from sklearn.model_selection import GridSearchCV
from gensim.models.coherencemodel import CoherenceModel

In [9]:
def scorer(estimator, X,y=None):
    goodcm = CoherenceModel(model=estimator, texts= texts, dictionary=estimator.id2word, coherence='c_v')
    return goodcm.get_coherence()

In [10]:
obj=SklearnWrapperLdaModel(id2word=dictionary,num_topics=5,passes=20)
parameters = {'num_topics':(2, 3, 5, 10), 'iterations':(1,20,50)}
model = GridSearchCV(obj, parameters, scoring=scorer, cv=5)
model.fit(corpus)

NameError: global name 'get_params' is not defined

In [60]:
model.best_params_

{'iterations': 20, 'num_topics': 3}